In [3]:
from ml_battery import *
import sklearn.pipeline
import pickle
import pandas as pd
import numpy as np
import sklearn.ensemble
import sklearn.svm
import sklearn.linear_model
import sklearn.cross_validation
import os.path
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
data_folder = os.path.join("..","data","data_from_jake")
df = pd.read_csv(os.path.join(data_folder, "links_2016toyotacamry_cv-ML-TESTING.csv"))
df = df.drop("orig_file", axis=1)
data_train, data_test = sklearn.cross_validation.train_test_split(df, test_size=0.2, random_state=0)

In [13]:
data_train.head()

,oid,sectionid,vsp_avg,cycGrade,cycSecs_start,cycSecs_end,speed_mph_start,speed_mph_end,speed_mph_avg,laneIndex,Length (m),Section Type,# of Main Lanes,# of Side Lanes,essKwhOutAch,gge
209758,15293,12226.0,84.407512,0,92,101,53.425626,58.346898,55.543265,1.0,244.831,Freeway Merge,4,1,0.0,0.005415
1328143,11578,12130.0,6.311837,0,23,34,23.724010,24.432375,22.378738,2.0,114.355,On Ramp,4,1,0.0,0.002227
809918,16679,12090.0,104.470778,0,23,38,17.951458,47.404525,35.778641,1.0,260.611,On Ramp,1,0,0.0,0.012704
17360,4679,12056.0,108.163650,0,18,23,32.249244,42.756657,37.895451,1.0,109.942,On Ramp,1,0,0.0,0.005166
133888,11049,12200.0,93.619535,0,488,517,65.654242,65.486471,65.628144,4.0,872.326,Freeway Mainline,4,0,0.0,0.013153


In [14]:
X_train, y_train = data_train.drop("gge", axis=1), data_train[["gge"]]
X_test, y_test = data_test.drop("gge", axis=1), data_test[["gge"]]
weights_train, weights_test = np.ones(X_train.shape[0]), np.ones(X_test.shape[0])

In [15]:
codebook = pd.DataFrame(
    {
        "Name": ["# of Main Lanes","# of Side Lanes","laneIndex","Section Type"],
        "Type": ["C"]*4
    }
).set_index("Name")

In [16]:
clf = sklearn.pipeline.Pipeline([
    ('data_transformation', CodebookTransformer(codebook)),
    ('feature_selection', SelectFromModelPandas(sklearn.ensemble.RandomForestRegressor())),
    ('classification', StackedRegressor(MultiRegressor([
            ("random forest", HyperparameterOptimizedRegressor(sklearn.ensemble.RandomForestRegressor(), max_depth=(2,30))),
            ("linear regression", sklearn.linear_model.LinearRegression()),
            ("ridge regression", sklearn.linear_model.Ridge()),
            ("dummy regressor", PatchedDummyRegressor()),
            ("one-hidden MLP", OneLayerNNRegressor())
        ], parallel=False), sklearn.linear_model.LinearRegression())),
])

In [17]:
clf.fit(X_train, y_train)

C:\ProgramData\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda\lib\site-packages\sklearn\feature_selection\from_model.py:169: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


Initialization
-------------------------------------------
 Step |   Time |      Value |   max_depth | 


C:\ProgramData\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    1 | 07m15s |    0.00000 |     23.5178 | 


C:\ProgramData\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    2 | 05m23s |    0.00000 |     15.5387 | 


C:\ProgramData\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [ ]:
multiclf = sklearn.pipeline.Pipeline(clf.steps[:2] + [("classifier", clf.steps[2][1].multiestimator)])

In [ ]:
pred_stacked = clf.predict(X_test)
pred = multiclf.predict(X_test)

In [ ]:
f, axes = plt.subplots(len(clf.steps[-1][1].multiestimator.estimators)+1, sharex=True, sharey=True, figsize=(12,40))
for i in range(len(axes)-1):
    ax = axes[i]
    ax.scatter(y_test, pred[i][1])
axes[-1].scatter(y_test, pred_stacked)


f.subplots_adjust(hspace=0)
plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)
plt.show()

In [ ]:
(stacked_score, stacked_feature_importances), (multi_scores, multi_feature_importances) = clf.score(X_test, y_test)
multi_scores["stacked"] = stacked_score

In [ ]:
multi_scores